## Prerequisites

Before delving into the fine-tuning process, ensure that you have the following prerequisites in place:

1. **GPU**: This tutorial cannot run on free Google Colab; it requires more powerful GPUs, such as the A100.
2. **Python Packages**: Ensure that you have the necessary Python packages installed. You can use the following commands to install them:

Let's begin by checking if your GPU is correctly detected:

Let's define a wrapper function which will get completion from the model from a user question

Let's define a wrapper function which will get completion from the model from a user question

## Step 1 Set up

### Step 1.1 - Set variables

In [43]:
MODEL_PATH = '/content/drive/My Drive/Colab Notebooks/model'
MODEL_NAME = 'llm-prompt-recovery'
CHECKPOINT_PATH = '/content/drive/My Drive/Colab Notebooks/checkpoints'

# TRAIN_FILE_PATH = '/content/drive/My Drive/Colab Notebooks/train_example.csv'
TRAIN_FILE_PATH = '/content/kaggle/df_with_emb_20240405.csv'
TEST_FILE_PATH = '/content/drive/My Drive/Colab Notebooks/test_example.csv'

HUGGING_FACE_USERNAME = 'valentinalilova'
HUGGING_FACE_REPO_PATH = '/content/drive/My Drive/Colab Notebooks/huggingFaceRepo'
# print(f'{HUGGING_FACE_USERNAME}/{MODEL_NAME}')

KAGGLE_CREDENTIALS_PATH = '/content/drive/My Drive/Colab Notebooks/kaggle.json'
KAGGLE_DATASET_PATH = '/content/dataset'
KAGGLE_DATASET_NAME = MODEL_NAME
KAGGLE_USERNAME = HUGGING_FACE_USERNAME

### Step 1.2 - Set Credentials

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from huggingface_hub import notebook_login
notebook_login()
# login()

In [5]:
# Setup Kaggle API Credentials:
# Make sure kaggle.json file exists in this folder
!mkdir ~/.kaggle
!cp "$KAGGLE_CREDENTIALS_PATH" ~/.kaggle/
!ls ~/.kaggle/

# Change the permissions of the file.
!chmod 600 ~/.kaggle/kaggle.json

kaggle.json


### Step 1.3 - Install necessary packages
First, install the dependencies below to get started. As these features are available on the main branches only, we need to install the libraries below from source.

In [6]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets scipy
!pip install -q trl

!pip install -q kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━

### Step 1.4 - Get the data

In [5]:
!kaggle kernels output tomooinubushi/all-in-one-dataset-with-embedding -p /content/kaggle

Output file downloaded to /content/kaggle/df_with_emb_20240405.csv
Output file downloaded to /content/kaggle/df_with_emb_20240405.parquet
Kernel log downloaded to /content/kaggle/all-in-one-dataset-with-embedding.log 


In [7]:
!ls /content/kaggle

all-in-one-dataset-with-embedding.log  df_with_emb_20240405.csv  df_with_emb_20240405.parquet


### Step 1.5 - Imports

In [44]:
import pandas as pd
import os
import torch
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftConfig, PeftModel
from datasets import Dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model
import transformers
from trl import SFTTrainer
import warnings
from huggingface_hub import Repository

## Step 2 - Model loading
We'll load the model using QLoRA quantization to reduce the usage of memory


In [9]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= 'nf4',
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
 )


model_id = 'mistralai/Mistral-7B-Instruct-v0.1'

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [10]:
adapter_model_name = 'dfurman/Mistral-7B-Instruct-v0.2'
model = PeftModel.from_pretrained(model, adapter_model_name)

adapter_config.json:   0%|          | 0.00/520 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

Now we specify the model ID and then we load it with our previously defined quantization configuration.

Run a inference on the base model. The model does not seem to understand our instruction and gives us a list of questions related to our query.

## Step 3 - Load dataset for finetuning

### Lets Load the Dataset

For this tutorial, we will fine-tune Mistral 7B Instruct for code generation.

We will be using this [dataset](https://huggingface.co/datasets/TokenBender/code_instructions_122k_alpaca_style) which is curated by [TokenBender (e/xperiments)](https://twitter.com/4evaBehindSOTA) and is an excellent data source for fine-tuning models for code generation. It follows the alpaca style of instructions, which is an excellent starting point for this task. The dataset structure should resemble the following:

```json
{
  "instruction": "Create a function to calculate the sum of a sequence of integers.",
  "input": "[1, 2, 3, 4, 5]",
  "output": "# Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum"
}
```

In [11]:
# df = pd.read_csv(TRAIN_FILE_PATH)

In [12]:
df = pd.read_csv(TRAIN_FILE_PATH)

def create_prompt(original, rewritten):
    return f"Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.\nYou are trying to understand how the original essay was transformed into a new version.\nAnalyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay.\nOnly give me the PROMPT. Start directly with the prompt, that's all I need.\nOutput should be only line ONLY\nOriginal Essay: {original}\nRewritten Essay: {rewritten}"

# Apply the function to each row to create the prompt column
df['prompt'] = df.apply(lambda row: create_prompt(row['original_text'], row['rewritten_text']), axis=1)

In [13]:
df.head(10)

,original_text,rewrite_prompt,rewritten_text,dataset_id,original_text_emb_0,original_text_emb_1,original_text_emb_2,original_text_emb_3,original_text_emb_4,original_text_emb_5,...,rewritten_text_emb_759,rewritten_text_emb_760,rewritten_text_emb_761,rewritten_text_emb_762,rewritten_text_emb_763,rewritten_text_emb_764,rewritten_text_emb_765,rewritten_text_emb_766,rewritten_text_emb_767,prompt
0,"Dear Randy,\n\nI hope this letter finds you we...",Rephrase this letter to infuse it with an elfi...,"Dear Randy,\n\nMay this enchanted message find...",host,-0.028678,-0.072260,-0.003482,0.050595,-0.010600,-0.023786,...,0.011079,-0.026521,-0.035838,-0.011306,0.035690,-0.005810,0.004506,-0.037956,0.001139,"Given are 2 essays, the Rewritten essay was cr..."
1,"This quilt, that my mother made, \n \n Still m...",Regency Romance: Model the text on a Regency r...,"The softest brown and brightest blue quilt, cr...",nbroad_1,-0.007521,-0.082697,0.041606,0.048020,0.007870,-0.018765,...,-0.019648,-0.044074,-0.049267,0.000438,0.047913,0.042219,0.012958,-0.048047,0.006332,"Given are 2 essays, the Rewritten essay was cr..."
2,It's the job of our agency to keep track of th...,Write like Ernest Hemingway: Focus on Hemingwa...,The agency's responsibility is to track and co...,nbroad_1,-0.012435,-0.057448,0.041290,0.021997,-0.005588,-0.011431,...,-0.024727,-0.005645,-0.049009,-0.007487,0.029255,0.023511,0.006769,-0.041317,-0.012102,"Given are 2 essays, the Rewritten essay was cr..."
3,"The first punch gets me right in the ribs, kno...",Grimm's Fairy Tales: Adapt the text to mimic t...,"In the sweltering sun, the stench of sweat and...",nbroad_1,0.023922,-0.051777,0.037086,0.057018,0.006423,0.006742,...,0.006210,-0.010690,-0.057320,0.015437,0.041056,0.012486,-0.006540,-0.030836,-0.000440,"Given are 2 essays, the Rewritten essay was cr..."
4,Some nights I lay awake staring at the ceiling...,High Fantasy Epic: Transform the essay into a ...,In the tapestry of the ethereal realm of Eldri...,nbroad_1,-0.002841,-0.056669,0.049056,0.072066,-0.006430,-0.020034,...,0.014793,-0.024928,-0.052689,-0.005655,0.044724,0.016817,-0.001000,-0.026472,0.008088,"Given are 2 essays, the Rewritten essay was cr..."
5,"I can hardly read the letter, because the hand...",Fairy Tale Villain: Use the menacing and craft...,"My hand quivered as I clutched the letter, the...",nbroad_1,-0.037955,-0.077311,0.039333,0.026843,-0.003027,-0.016087,...,-0.021211,0.001880,-0.032622,0.013712,0.018871,0.021326,0.011081,-0.025262,-0.015825,"Given are 2 essays, the Rewritten essay was cr..."
6,`` They do n't believe we're interesting?'' on...,"Beat Generation: Channel the spontaneous, free...",The mermaids' council deliberated on the dwind...,nbroad_1,-0.024788,-0.072080,0.046875,0.012351,-0.012402,-0.016356,...,0.013847,-0.006507,-0.027821,-0.012744,0.035653,0.029916,-0.011108,-0.007995,-0.009567,"Given are 2 essays, the Rewritten essay was cr..."
7,Not a single person in the crowd of Nora Janic...,"Fantasy Dwarf: Write with the gruff, hearty st...",The crowd at Nora Janice's funeral was silent ...,nbroad_1,-0.019732,-0.068090,0.044369,0.022035,-0.013128,-0.012057,...,-0.002342,-0.000681,-0.049935,-0.018382,0.035788,0.023763,0.022539,-0.046245,-0.007326,"Given are 2 essays, the Rewritten essay was cr..."
8,`` Brigands and cutpurses have nothing on me. ...,"Drunkard: Infuse the essay with the rambling, ...","""Swerry brutes and cutthroat cutpurse, they ai...",nbroad_1,-0.013881,-0.065521,0.003461,0.035074,0.014662,-0.009928,...,-0.027956,-0.042242,-0.058649,0.016741,0.045051,0.022986,-0.016166,-0.027737,-0.035562,"Given are 2 essays, the Rewritten essay was cr..."
9,Sergeant Clark lifted his wrist to look at the...,High Fantasy Epic: Transform the essay into a ...,The scent of ash and molten earth hung heavy i...,nbroad_1,-0.014956,-0.083447,0.047591,0.024411,0.021683,0.004255,...,0.006138,-0.025443,-0.060491,0.008632,0.040641,0.024933,0.000982,-0.009748,0.011877,"Given are 2 essays, the Rewritte

Instruction Fintuning - Prepare the dataset under the format of "prompt" so the model can better understand :
1. the function generate_prompt : take the instruction and output and generate a prompt
2. shuffle the dataset
3. tokenizer the dataset

### Formatting the Dataset

Now, let's format the dataset in the required [Mistral-7B-Instruct-v0.1 format](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1).

> Many tutorials and blogs skip over this part, but I feel this is a really important step.

We'll put each instruction and input pair between `[INST]` and `[/INST]` output after that, like this:

```
<s>[INST] What is your favorite condiment? [/INST]
Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavor to whatever I'm cooking up in the kitchen!</s>
```

You can use the following code to process your dataset and create a JSONL file in the correct format:

In [14]:
def generate_prompt(row):
    """Generate input text based on a prompt, task instruction, and answer.

    :param row: Series: Data point (row of a pandas DataFrame)
    :return: str: generated prompt text
    """
    # Assuming 'prompt' is the generated context and 'rewrite_prompt' is the instruction
    return f"<s>[INST]{row['prompt']} [/INST]{row['rewrite_prompt']}</s>"

# Apply the function to each row to transform the 'prompt' and 'rewrite_prompt' columns
df['formatted_prompt'] = df.apply(generate_prompt, axis=1)

In [15]:
# import numpy as np
# import pandas as pd

# #Reduce Memory Usage
# def reduce_memory_usage(df):

#     start_mem = df.memory_usage().sum() / 1024**2
#     print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

#     for col in df.columns:
#         col_type = df[col].dtype.name
#         if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
#             if (col_type != 'object'):
#                 c_min = df[col].min()
#                 c_max = df[col].max()

#                 if str(col_type)[:3] == 'int':
#                     if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
#                         df[col] = df[col].astype(np.int8)
#                     elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                         df[col] = df[col].astype(np.int16)
#                     elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                         df[col] = df[col].astype(np.int32)
#                     elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                         df[col] = df[col].astype(np.int64)

#                 else:
#                     if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                         df[col] = df[col].astype(np.float16)
#                     elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                         df[col] = df[col].astype(np.float32)
#                     else:
#                         pass
#             else:
#                 df[col] = df[col].astype('category')
#     mem_usg = df.memory_usage().sum() / 1024**2
#     print("Memory usage became: ",mem_usg," MB")

#     return df

# df.info()
# df = reduce_memory_usage(df)
# df.info()


In [16]:
df = df[['prompt', 'rewrite_prompt']]

# # ToDo
# # Converting back to strings
# for col in df.select_dtypes(['category']).columns:
#     df[col] = df[col].astype(str)

# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.select(range(1000)) # ToDo: remove that
# Shuffle the dataset with a seed for reproducibility
dataset = dataset.shuffle(seed=1234)

# Tokenize the prompts in the dataset.
# This example assumes that the model requires only 'input_ids'.
# Adjust as necessary for your model, e.g., adding 'attention_mask'.
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

# Split the dataset into training and testing sets (90% training, 10% testing)
train_test_split = dataset.train_test_split(test_size=0.1)
train_data = train_test_split['train']
test_data = train_test_split['test']

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [17]:

# dataset.info()
# dataset = reduce_memory_usage(dataset)
# dataset.info()

We'll need to tokenize our data so the model can understand.


Split dataset into 90% for training and 10% for testing

### After Formatting, We should get something like this

```json
{
"text":"<s>[INST] Create a function to calculate the sum of a sequence of integers. here are the inputs [1, 2, 3, 4, 5] [/INST]
# Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum</s>",
"instruction":"Create a function to calculate the sum of a sequence of integers",
"input":"[1, 2, 3, 4, 5]",
"output":"# Python code def sum_sequence(sequence): sum = 0 for num in,
 sequence: sum += num return sum"
"prompt":"<s>[INST] Create a function to calculate the sum of a sequence of integers. here are the inputs [1, 2, 3, 4, 5] [/INST]
# Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum</s>"

}
```

While using SFT (**[Supervised Fine-tuning Trainer](https://huggingface.co/docs/trl/main/en/sft_trainer)**) for fine-tuning, we will be only passing in the “text” column of the dataset for fine-tuning.

In [18]:
print(test_data)

Dataset({
    features: ['prompt', 'rewrite_prompt', 'input_ids', 'attention_mask'],
    num_rows: 100
})


## Step 4 - Apply Lora  
Here comes the magic with peft! Let's load a PeftModel and specify that we are going to use low-rank adapters (LoRA) using get_peft_model utility function and  the prepare_model_for_kbit_training method from PEFT.

In [19]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [20]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): 

Use the following function to find out the linear layers for fine tuning.
QLoRA paper : "We find that the most critical LoRA hyperparameter is how many LoRA adapters are used in total and that LoRA on all linear transformer block layers is required to match full finetuning performance."

In [21]:
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [22]:
modules = find_all_linear_names(model)
print(modules)

['up_proj', 'base_layer', 'gate_proj', 'down_proj']


In [23]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)

model = get_peft_model(model, lora_config)

In [24]:
# trainable, total = model.get_nb_trainable_parameters()
# print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")


## Step 5 - Run the training!

Setting the training arguments:
* for the reason of demo, we just ran it for few steps (100) just to showcase how to use this integration with existing tools on the HF ecosystem.

In [25]:
# from datasets import load_dataset
# data = load_dataset("TokenBender/code_instructions_122k_alpaca_style", split='train')
# data = data.train_test_split(test_size=0.1)
# train_data = data["train"]
# test_data = data["test"]

In [ ]:
# import transformers
# tokenizer.pad_token = tokenizer.eos_token

# trainer = transformers.Trainer(
#      model=model,
#      train_dataset=train_data,
#      eval_dataset=test_data,
#      args=transformers.TrainingArguments(
#          per_device_train_batch_size=1,
#          gradient_accumulation_steps=4,
#          warmup_ratio=0.03,
#          max_steps=100,
#          learning_rate=2e-4,
#          fp16=True,
#          logging_steps=1,
#          output_dir="outputs_mistral_b_finance_finetuned_test",
#          optim="paged_adamw_8bit",
#          save_strategy="epoch",
#      ),
#      data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
#  )


### Fine-Tuning with qLora and Supervised Fine-Tuning

We're ready to fine-tune our model using qLora. For this tutorial, we'll use the `SFTTrainer` from the `trl` library for supervised fine-tuning. Ensure that you've installed the `trl` library as mentioned in the prerequisites.

In [36]:
#new code using SFTTrainer
tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=4, # This was 1. ToDo: Fix memory issues and increase to 64
    gradient_accumulation_steps=4,
    warmup_steps=0.03,
    max_steps=100, # ToDo: Run with 2100
    learning_rate=2e-4,
    logging_steps=1,
    # output_dir='outputs',
    optim='paged_adamw_8bit',
    save_strategy='steps',
    save_steps=2,
    output_dir=CHECKPOINT_PATH,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field='prompt',
    peft_config=lora_config,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Start the training

### Let's start the training process

In [45]:
def find_latest_checkpoint(checkpoint_path):
    """
    Finds the latest (highest-numbered) checkpoint directory.

    Parameters:
    checkpoint_path (str): The base path where checkpoint directories are stored.

    Returns:
    str: The path to the latest checkpoint directory, or None if no checkpoint found.
    """
    # Regex to match checkpoint directories like 'checkpoint-500'
    checkpoint_pattern = re.compile(r"^checkpoint-\d+$")

    # Get all directories in the checkpoint path
    all_files = os.listdir(checkpoint_path)

    # Filter out checkpoint directories
    checkpoint_dirs = [f for f in all_files if checkpoint_pattern.match(f)]

    # No checkpoint directories found
    if not checkpoint_dirs:
        return None

    # Find the checkpoint directory with the highest step number
    latest_checkpoint_dir = max(checkpoint_dirs, key=lambda x: int(x.split('-')[-1]))

    # Return the full path to the latest checkpoint directory
    return os.path.join(checkpoint_path, latest_checkpoint_dir)


In [46]:
# Suppress all warnings
warnings.filterwarnings('ignore')

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

# Find the latest checkpoint
latest_checkpoint_path = find_latest_checkpoint(CHECKPOINT_PATH)

if latest_checkpoint_path and os.path.isdir(latest_checkpoint_path):
    print(f"Resuming training from the latest checkpoint: {latest_checkpoint_path}")
    trainer.train(resume_from_checkpoint=latest_checkpoint_path)
else:
    print(f"No checkpoints found in: {CHECKPOINT_PATH}. Training will start from scratch.")
    trainer.train()

Resuming training from the latest checkpoint: /content/drive/My Drive/Colab Notebooks/checkpoints/checkpoint-6


KeyboardInterrupt: 

## Step 6 - Save the model

In [ ]:
# Save the model to the repo
model.save_pretrained(MODEL_PATH)
tokenizer.save_pretrained(MODEL_PATH)

In [14]:
# # Or load from the file
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit= True,
#     bnb_4bit_quant_type= 'nf4',
#     bnb_4bit_compute_dtype= torch.bfloat16,
#     bnb_4bit_use_double_quant= False,
#  )

# model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, quantization_config=bnb_config, device_map={"":0})
# tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, add_eos_token=True)


NameError: name 'model' is not defined

### Save to Hugging Face

In [ ]:
# # Create repo if needed
# from huggingface_hub import create_repo
# create_repo(MODEL_NAME, private=True)

In [ ]:
# Clone the repo
repo = Repository(HUGGING_FACE_REPO_PATH, clone_from=f"{HUGGING_FACE_USERNAME}/{MODEL_NAME}")

In [ ]:
# Move the model files to the repo
!cp "$MODEL_PATH"/* "$HUGGING_FACE_REPO_PATH"/
!ls $HUGGING_FACE_REPO_PATH

In [ ]:
# # Debugging
# %cd $HUGGING_FACE_REPO_PATH
# !git status
# # %cd /content

In [ ]:
# # Debugging
# !cat .git/hooks/pre-push

In [ ]:
# # Fix push loading forever
# !chmod +x .git/hooks/pre-push
# !git lfs install

In [ ]:
repo.push_to_hub(commit_message='Upload model')
# # Other ways to push
# repo.git_push()
# !git push --verbose

### Save to Kaggle


In [ ]:
# Ensure the directory exists
os.makedirs(KAGGLE_DATASET_PATH, exist_ok=True)

# Move the model files to the directory
!cp "$MODEL_PATH"/* "$KAGGLE_DATASET_PATH"/
!ls $KAGGLE_DATASET_PATH


In [ ]:
# Add a metadata file
metadata = {
    'title': KAGGLE_DATASET_NAME,
    'id': f"{KAGGLE_USERNAME}/{KAGGLE_DATASET_NAME}",
    'licenses': [{'name': 'CC0-1.0'}] # ToDo: Use that license? What metadata?
}

with open(os.path.join(KAGGLE_DATASET_PATH, 'dataset-metadata.json'), 'w') as f:
    json.dump(metadata, f)

In [ ]:
# Add the test CSV file
# ToDo: Add the real test CSV or remove
!cp "$TEST_FILE_PATH" "$KAGGLE_DATASET_PATH"

In [ ]:
# Upload the dataset to Kaggle
# Create the first time
# !kaggle datasets create -p {KAGGLE_DATASET_PATH} --dir-mode zip
# Update an existing dataset
!kaggle datasets version -p {KAGGLE_DATASET_PATH} -m "Update dataset"

## Step 7 - Evaluating the model qualitatively: run an inference!



In [ ]:
# import re
# def remove_special_characters(text):
#     # This regex will match any character that is not a letter, number, or whitespace
#     pattern = r'[^a-zA-Z0-9\s]'
#     text =text.replace("Transform" ,"improve")
#     text =text.replace("Reimagine" ,"rewrite")
#     # Replace these characters with an empty string
#     clean_text = re.sub(pattern, '', text)
#     return clean_text

# def get_completion(prompt) -> str:
#     device = "cuda:0"
#     print('prompt')
#     print(prompt)
#     encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

#     model_inputs = encodeds.to(device)


#     generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True, pad_token_id=tokenizer.eos_token_id)
#     generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
#     print('gen')
#     print(generated_text)
#     prompt_length = len(tokenizer.encode(prompt))
#     print('output')
#     prompt_tokens = tokenizer.encode(prompt, add_special_tokens=False)
#     generated_tokens = generated_ids[0][len(prompt_tokens):]
#     generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
#     print(generated_text)
#     return generated_text

In [ ]:
# test_df = pd.read_csv("/kaggle/input/llm-prompt-recovery/test.csv")
# test_df['prompt'] = test_df.apply(lambda row: create_prompt(row['original_text'], row['rewritten_text']), axis=1)
# test_df['prompt'] = test_df.apply(generate_prompt, axis=1)
# test_df['rewrite_prompt'] = test_df.apply(lambda row: get_completion(row['prompt']), axis=1)
# test_df = test_df[['id', 'rewrite_prompt']]
# test_df

In [ ]:
# test_df = pd.read_csv("/kaggle/input/llm-prompt-recovery/test.csv")
# adject =[]
# for index, row in test_df.iterrows():
#         result = generate_prompt(create_prompt(row['original_text'], row['rewritten_text']))
#         result  = remove_special_characters(result)
#         print(result)
#         test_df.at[index, 'rewrite_prompt'] =  result




# test_df = test_df[['id', 'rewrite_prompt']]
# test_df


In [ ]:
# test_df.to_csv('submission.csv', index=False)